In [1]:
# %pip install flask_cors

In [2]:
import json
import numpy as np
from ultralytics import YOLO
import cv2
import cvzone
import math
from sort import *

In [3]:
from flask import Flask, request, jsonify,send_file,render_template
from flask_cors import CORS
import os
import subprocess


app = Flask(__name__)
CORS(app)



In [4]:
from flask import Flask, request, jsonify,send_file,render_template
from flask_cors import CORS
import os

app = Flask(__name__)
CORS(app)


@app.route('/process_video', methods=['POST'])
def process_video():
    print("Got request !!!")
    uploaded_video = request.files['file']
    if uploaded_video.filename != '':
        # uploaded_video.save('uploads/' + uploaded_video.filename)
        print(uploaded_video.filename)
        
        output={}

        frame_width = 1280
        frame_height = 720


        def keyss(allkeys,x1, y1, x2, y2):
            for k in allkeys:
                all_within_box = all(
                    x1 <= x <= x2  and
                    y1 <= y <= y2 
                    for x, y in k
                )

                if all_within_box:
                    return k.tolist()
            return 0


        
        i=0
        res={}

        cap = cv2.VideoCapture(str("C:/Users/Rajesh/Downloads/"+uploaded_video.filename))


        wCam, hCam = 640, 480
        cap.set(3, wCam)
        cap.set(4, hCam)

        model = YOLO("C:/OpenCV/yolov8n-pose.pt")

        classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
                    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
                    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
                    "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
                    "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
                    "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
                    "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
                    "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
                    "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
                    "teddy bear", "hair drier", "toothbrush"
                    ]

        # mask = cv2.imread("mask.png")

        output_path = 'input.mp4'

        frame_width = int(cap.get(3))  # Width
        frame_height = int(cap.get(4))  # Height

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, 10, (frame_width, frame_height))



        # Tracking

        tracker = Sort(max_age=30, min_hits=3, iou_threshold=0.3)

        frame_rate = 25

        frame_skip = int(24 / frame_rate)  # Calculate the number of frames to skip

        frame_number = 0 

        while True:

            dummy={}
            success, img = cap.read()

            if frame_number % 3 == 0:
                frame_number += 1
                continue
            else:
                frame_number += 1
            if not success:
                print("Ignoring empty camera  frame.",i)
            # If loading a video, use 'break' instead of 'continue'.
                break
            # imgRegion = cv2.bitwise_and(img, mask)

            # imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
            # img = cvzone.overlayPNG(img, imgGraphics, (730, 260))
            results = model(img)

            # print(results)

            all_keypoints=[]  
            detections = np.empty((0, 5))

            for r in results:
                keypoints = r.keypoints
                for k in keypoints:
                    # print("Key :" ,k.xy[0])
                    all_keypoints.append(k.xy[0])
                boxes = r.boxes
                for box in boxes:
                    # print("Box :", box.xyxy[0])
                    # Bounding Box
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                    # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
                    w, h = x2 - x1, y2 - y1

                    # Confidence
                    conf = math.ceil((box.conf[0] * 100)) / 100
                    # Class Name
                    cls = int(box.cls[0])
                    currentClass = classNames[cls]

                    if currentClass == "person" and conf > 0.3:
                        # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                        #                    scale=0.6, thickness=1, offset=3)
                        # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                        currentArray = np.array([x1, y1, x2, y2, conf])
                        detections = np.vstack((detections, currentArray))


            # print("ALL Keypoints :",all_keypoints)
            resultsTracker = tracker.update(detections)
            img = results[0].plot()
            image = img
            
            for result in resultsTracker:
                x1, y1, x2, y2, id = result
                # print(result)
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2 - x1, y2 - y1
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
                cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                                scale=2, thickness=3, offset=10)

                # cx, cy = x1 + w // 2, y1 + h // 2
                # cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

                # label=input("Enter "+str(id)+" :")
                a = {
                    "person_id" : id ,
                    "box" : [x1, y1, x2, y2] ,
                    "keypoints" : keyss(all_keypoints,x1, y1, x2, y2),
                    "label" : 0
                }
                dummy[str(id)] = a
            
            res[str(i)]=dummy   
            i+=1

            # Flip the image horizontally for a selfie-view display.
            # frame_resized = cv2.resize(img, (frame_width, frame_height))

            cv2.imshow("Image", img)
            out.write(img)

            # cv2.imshow("Image22", image)
            # cv2.imshow("ImageRegion", imgRegion)
            key = cv2.waitKey(1)
            if key == ord('q'):
                break
        # with open('output15.json', 'w') as json_file:
        #     json.dump(res, json_file, indent=2)

        cap.release()
        out.release()
        cv2.destroyAllWindows()
        output['message']=res
        # resultsTracker = tracker.update(0)


        # Define the command as a list of strings
        command = ['ffmpeg', '-y', '-i', 'C:/OpenCV/consult/input.mp4', '-vcodec', 'libx264', '-acodec', 'aac', 'C:/OpenCV/consult/output.mp4']

        # Run the command
        subprocess.run(command)
     
        tracker.reset()

        

        
        return (output)


    return jsonify(message="No video uploaded.")



In [5]:
@app.route('/video', methods=['GET'])
def video():
    return send_file('C:/OpenCV/consult/output.mp4', mimetype='video/mp4')

In [6]:
if __name__ == '__main__':
    
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Got request !!!
20230916040916.mp4



0: 448x640 (no detections), 163.1ms
Speed: 4.3ms preprocess, 163.1ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 148.3ms
Speed: 3.8ms preprocess, 148.3ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 145.7ms
Speed: 7.5ms preprocess, 145.7ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 148.2ms
Speed: 3.5ms preprocess, 148.2ms inference, 0.2ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 146.8ms
Speed: 3.2ms preprocess, 146.8ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 131.5ms
Speed: 2.0ms preprocess, 131.5ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 139.2ms
Speed: 2.0ms preprocess, 139.2ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 137.8ms
Speed: 2.9ms prepr

Ignoring empty camera  frame. 176


...


0: 448x640 (no detections), 194.6ms
Speed: 2.0ms preprocess, 194.6ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 (no detections), 130.3ms
Speed: 2.5ms preprocess, 130.3ms inference, 0.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 131.9ms
Speed: 2.0ms preprocess, 131.9ms inference, 0.3ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 132.2ms
Speed: 2.0ms preprocess, 132.2ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 132.3ms
Speed: 2.7ms preprocess, 132.3ms inference, 1.6ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 145.6ms
Speed: 6.0ms preprocess, 145.6ms inference, 1.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 131.6ms
Speed: 2.4ms preprocess, 131.6ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 person, 124.2ms
Speed: 2.6ms preprocess, 124.2ms inference, 2.0ms postproces

Ignoring empty camera  frame. 410


127.0.0.1 - - [18/Oct/2023 16:06:22] "POST /process_video HTTP/1.1" 200 -
127.0.0.1 - - [18/Oct/2023 16:06:22] "GET /video HTTP/1.1" 200 -


Got request !!!
demo6.mp4



0: 384x640 2 persons, 134.4ms
Speed: 1.0ms preprocess, 134.4ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 98.5ms
Speed: 0.0ms preprocess, 98.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 81.1ms
Speed: 3.8ms preprocess, 81.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 93.5ms
Speed: 7.0ms preprocess, 93.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 89.5ms
Speed: 0.0ms preprocess, 89.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 85.5ms
Speed: 3.4ms preprocess, 85.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 91.9ms
Speed: 0.0ms preprocess, 91.9ms inference, 5.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 82.4ms
Speed: 2.4ms preprocess, 82.4ms inference, 0.0ms postprocess per image at shape